# 🚀 단일 모델 실험 - SOLAR/Polyglot-Ko LLM 파인튜닝
> PRD 계획에 따른 LLM 모델 LoRA 파인튜닝

**목표 성능**: ROUGE-F1 70-73

In [1]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent.parent  # 3번만 parent 사용!

# 다른 프로젝트 경로 제거하고 현재 프로젝트 경로만 추가
sys.path = [p for p in sys.path if 'computer-vision-competition' not in p]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project Root: {project_root}")
print(f"Current Dir: {notebook_dir}")

# 필요한 라이브러리 임포트
import yaml
import pandas as pd
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import wandb

# 커스텀 모듈 임포트
from src.logging.notebook_logger import NotebookLogger
from src.utils.gpu_optimization.team_gpu_check import check_gpu_tier

print("Libraries imported successfully!")

Project Root: /home/ieyeppo/AI_Lab/natural-language-processing-competition
Current Dir: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH
Libraries imported successfully!


In [2]:
# 설정 파일 로드
config_path = notebook_dir / 'configs' / 'config_single_model.yaml'

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# 모델 선택
current_model = config['current_model']
model_config = config['models'][current_model]

print(f"Selected Model: {model_config['name']}")
print(f"Using LoRA: {model_config['use_lora']}")
if model_config['use_lora']:
    print(f"  - LoRA r: {model_config['lora_r']}")
    print(f"  - LoRA alpha: {model_config['lora_alpha']}")

Selected Model: upstage/SOLAR-10.7B-Instruct-v1.0
Using LoRA: True
  - LoRA r: 16
  - LoRA alpha: 32


In [3]:
# 로그 디렉토리 생성
log_dir = Path(config['paths']['log_dir'])
print(f"Log Directory: {log_dir}")
log_dir.mkdir(parents=True, exist_ok=True)

# 타임스탬프 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 로거 초기화
log_file = log_dir / f'single_model_{current_model}_{timestamp}.log'
logger = NotebookLogger(
    log_path=str(log_file),
    print_also=True
)

logger.write('='*50)
logger.write(f'Single Model Experiment: {current_model}')
logger.write(f'Timestamp: {timestamp}')
logger.write(f'Model: {model_config["name"]}')
logger.write('='*50)

Log Directory: logs/single_model
Single Model Experiment: solar
Timestamp: 20251010_090241
Model: upstage/SOLAR-10.7B-Instruct-v1.0


In [4]:
# GPU 체크
if torch.cuda.is_available():
    gpu_tier = check_gpu_tier()
    device = torch.device('cuda')
    logger.write(f"GPU: {torch.cuda.get_device_name(0)}")
    logger.write(f"GPU Tier: {gpu_tier}")
else:
    device = torch.device('cpu')
    logger.write("WARNING: No GPU available")

GPU: NVIDIA GeForce RTX 4090
GPU Tier: LOW


In [5]:
# LoRA 설정
if model_config['use_lora']:
    lora_config = LoraConfig(
        r=model_config['lora_r'],
        lora_alpha=model_config['lora_alpha'],
        lora_dropout=model_config['lora_dropout'],
        task_type=TaskType.CAUSAL_LM,
        target_modules=config['peft']['target_modules']
    )
    logger.write("LoRA configuration created")
    logger.write(f"  - r: {model_config['lora_r']}")
    logger.write(f"  - alpha: {model_config['lora_alpha']}")
    logger.write(f"  - dropout: {model_config['lora_dropout']}")

LoRA configuration created
  - r: 16
  - alpha: 32
  - dropout: 0.1


In [6]:
# 프롬프트 템플릿 확인
sample_dialogue = "#Person1#: 안녕하세요?\n#Person2#: 안녕하세요!"
prompt = config['prompt_template']['instruction_format'].format(
    dialogue=sample_dialogue,
    summary=""
)
logger.write("Prompt template loaded")
print("\nSample Prompt:")
print(prompt[:300] + "...")

Prompt template loaded

Sample Prompt:
### Instruction:
다음 대화를 3-5문장으로 요약해주세요. 핵심 내용과 중요한 정보를 포함시켜주세요.

### Input:
#Person1#: 안녕하세요?
#Person2#: 안녕하세요!

### Response:

...


In [ ]:
# 향상된 프롬프트 엔지니어링 (PRD 14_프롬프트_엔지니어링.md)
import requests
import json
from typing import Optional, Dict, List

class AdvancedPromptEngineering:
    """고급 프롬프트 엔지니어링 클래스"""
    
    def __init__(self):
        self.prompt_templates = {
            'few_shot': self._create_few_shot_template,
            'chain_of_thought': self._create_cot_template,
            'role_based': self._create_role_based_template,
            'structured': self._create_structured_template,
            'topic_specific': self._create_topic_specific_template
        }
        
    def _create_few_shot_template(self, dialogue: str, topic: str = None) -> str:
        """Few-shot 학습 프롬프트"""
        examples = """
예시 1:
대화: 화자1: 오늘 날씨가 정말 좋네요. 화자2: 네, 산책하기 좋은 날씨입니다.
요약: 두 사람이 좋은 날씨에 대해 이야기하며 산책하기 좋다고 동의합니다.

예시 2:
대화: 화자1: 프로젝트 마감일이 언제죠? 화자2: 다음 주 금요일입니다. 화자1: 서둘러야겠네요.
요약: 프로젝트 마감일이 다음 주 금요일이며, 서둘러 완성해야 한다고 대화합니다.

예시 3:
대화: 화자1: 감기 증상이 있어요. 화자2: 충분한 휴식을 취하세요. 약도 처방해드리겠습니다.
요약: 환자가 감기 증상을 호소하자 의사가 휴식과 약 처방을 권합니다.
"""
        
        return f"""### Instruction:
아래 예시를 참고하여 대화를 간결하고 정확하게 요약해주세요.

{examples}

### Input:
{dialogue}

### Response:"""
    
    def _create_cot_template(self, dialogue: str, topic: str = None) -> str:
        """Chain-of-Thought 프롬프트"""
        return f"""### Instruction:
다음 단계를 따라 대화를 분석하고 요약해주세요:

1단계: 대화 참여자들을 파악하세요
2단계: 대화의 주요 주제를 찾으세요
3단계: 핵심 정보와 결론을 추출하세요
4단계: 3-5문장으로 간결하게 요약하세요

### Input:
{dialogue}

### Analysis and Response:
1단계 분석:
2단계 분석:
3단계 분석:
4단계 최종 요약:"""
    
    def _create_role_based_template(self, dialogue: str, topic: str = None) -> str:
        """역할 기반 프롬프트"""
        role_description = {
            '의료': '의료 전문가로서 환자의 증상과 치료 계획을 명확히',
            '비즈니스': '비즈니스 분석가로서 주요 결정사항과 액션아이템을',
            '교육': '교육 전문가로서 학습 내용과 핵심 개념을',
            '기술': '기술 전문가로서 기술적 논의사항과 해결책을'
        }.get(topic, '전문 요약가로서 핵심 내용을')
        
        return f"""### Instruction:
당신은 {role_description} 요약하는 전문가입니다.
대화의 중요한 정보를 빠짐없이 포함하여 3-5문장으로 요약해주세요.

### Input:
{dialogue}

### Expert Summary:"""
    
    def _create_structured_template(self, dialogue: str, topic: str = None) -> str:
        """구조화된 출력 프롬프트"""
        return f"""### Instruction:
다음 형식에 맞춰 대화를 요약해주세요:

[주제]: 대화의 주요 주제
[참여자]: 대화 참여자들의 역할
[핵심내용]: 가장 중요한 정보 2-3가지
[결론/결과]: 대화의 결론이나 합의사항

### Input:
{dialogue}

### Structured Summary:"""
    
    def _create_topic_specific_template(self, dialogue: str, topic: str = None) -> str:
        """주제별 맞춤 프롬프트"""
        topic_instructions = {
            '건강검진': '환자의 건강 상태, 검진 항목, 의사의 권고사항을 중심으로',
            '백신 접종': '백신 종류, 접종 일정, 부작용 설명을 포함하여',
            '회의': '회의 안건, 논의사항, 결정사항을 명확히',
            '상담': '상담 목적, 문제점, 해결방안을 구체적으로',
            '예약': '예약 일시, 장소, 목적을 정확히'
        }
        
        instruction = topic_instructions.get(topic, '핵심 내용을 중심으로')
        
        return f"""### Instruction:
{instruction} 대화를 요약해주세요.
불필요한 세부사항은 제외하고 중요한 정보만 포함시켜주세요.

### Input:
{dialogue}

### Response:"""
    
    def get_best_prompt(self, dialogue: str, topic: str = None, strategy: str = 'auto') -> str:
        """최적 프롬프트 선택"""
        if strategy == 'auto':
            # 대화 길이에 따라 전략 선택
            if len(dialogue) < 500:
                strategy = 'structured'
            elif len(dialogue) < 1000:
                strategy = 'few_shot'
            elif topic:
                strategy = 'topic_specific'
            else:
                strategy = 'chain_of_thought'
        
        if strategy in self.prompt_templates:
            return self.prompt_templates[strategy](dialogue, topic)
        else:
            # 기본 템플릿
            return self.prompt_templates['few_shot'](dialogue, topic)

# Solar API 교차 검증 시스템 (PRD 09_Solar_API_최적화.md, 10_교차_검증_시스템.md)
class SolarAPIValidator:
    """Solar API 교차 검증 클래스"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.upstage.ai/v1/solar"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        self.cache = {}  # API 응답 캐싱
        
    def generate_with_solar(self, dialogue: str, max_tokens: int = 150, temperature: float = 0.3) -> Optional[str]:
        """Solar API로 요약 생성"""
        # 캐시 확인
        cache_key = hash(dialogue[:200])
        if cache_key in self.cache:
            logger.write("Using cached Solar API response")
            return self.cache[cache_key]
        
        try:
            # 프롬프트 최적화 (토큰 절약)
            if len(dialogue) > 2000:
                dialogue = dialogue[:2000] + "..."
            
            prompt = f"""다음 대화를 한국어로 간결하게 요약하세요:

{dialogue}

요약:"""
            
            payload = {
                "model": "solar-1-mini-chat",
                "messages": [
                    {"role": "system", "content": "당신은 전문적인 대화 요약 AI입니다."},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": max_tokens,
                "temperature": temperature,
                "top_p": 0.9
            }
            
            response = requests.post(
                f"{self.base_url}/chat/completions",
                headers=self.headers,
                json=payload,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                summary = result['choices'][0]['message']['content']
                
                # 캐싱
                self.cache[cache_key] = summary
                
                return summary
            else:
                logger.write(f"Solar API error: {response.status_code}")
                return None
                
        except Exception as e:
            logger.write(f"Solar API exception: {e}")
            return None
    
    def compare_with_model(self, model_summary: str, api_summary: str, reference: str = None) -> Dict:
        """모델과 API 요약 비교"""
        from rouge import Rouge
        
        result = {
            'model_summary': model_summary,
            'api_summary': api_summary
        }
        
        if reference:
            rouge = Rouge()
            try:
                # 모델 점수
                model_scores = rouge.get_scores(model_summary, reference)[0]
                result['model_rouge_l'] = model_scores['rouge-l']['f']
                
                # API 점수
                if api_summary:
                    api_scores = rouge.get_scores(api_summary, reference)[0]
                    result['api_rouge_l'] = api_scores['rouge-l']['f']
                else:
                    result['api_rouge_l'] = 0
                
                # 최선 선택
                result['best'] = 'model' if result['model_rouge_l'] >= result.get('api_rouge_l', 0) else 'api'
                result['improvement'] = abs(result['model_rouge_l'] - result.get('api_rouge_l', 0))
                
            except Exception as e:
                logger.write(f"Error computing ROUGE: {e}")
        
        return result

# 프롬프트 엔지니어링 초기화
prompt_engineer = AdvancedPromptEngineering()

# Solar API 초기화 (config에서 키 가져오기)
solar_validator = None
if 'solar_api' in config and 'api_key' in config['solar_api']:
    solar_validator = SolarAPIValidator(config['solar_api']['api_key'])
    logger.write("Solar API validator initialized for cross-validation")
else:
    logger.write("Solar API key not found - skipping API cross-validation")

# 프롬프트 테스트
sample_dialogue = train_df['dialogue'].iloc[0]
sample_topic = train_df.get('topic', pd.Series([None] * len(train_df))).iloc[0]

# 다양한 프롬프트 전략 테스트
logger.write("\n=== Testing Prompt Engineering Strategies ===")
for strategy in ['few_shot', 'chain_of_thought', 'topic_specific']:
    test_prompt = prompt_engineer.prompt_templates[strategy](sample_dialogue[:500], sample_topic)
    logger.write(f"\n{strategy.upper()} Strategy (first 300 chars):")
    logger.write(test_prompt[:300] + "...")

In [7]:
# 데이터 경로 설정 및 로드
# config 파일의 경로 사용
def get_data_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에서 데이터 경로 가져오기
train_path = get_data_path(config['paths']['train_file'])
dev_path = get_data_path(config['paths']['dev_file'])
test_path = get_data_path(config['paths']['test_file'])

logger.write(f"Loading data from config paths:")
logger.write(f"  - Train: {train_path}")
logger.write(f"  - Dev: {dev_path}")
logger.write(f"  - Test: {test_path}")

# 데이터 로드
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

logger.write(f"Data loaded successfully!")
logger.write(f"Train samples: {len(train_df)}")
logger.write(f"Dev samples: {len(dev_df)}")
logger.write(f"Test samples: {len(test_df)}")

# 데이터 샘플 출력
print("\nSample train data:")
print(train_df[['fname', 'topic']].head(3))
print(f"\nDialogue sample (first 200 chars):")
print(train_df.iloc[0]['dialogue'][:200] + "...")

Loading data from config paths:
  - Train: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/train.csv
  - Dev: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/dev.csv
  - Test: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/test.csv
Data loaded successfully!
Train samples: 12457
Dev samples: 499
Test samples: 499

Sample train data:
     fname  topic
0  train_0   건강검진
1  train_1  백신 접종
2  train_2  열쇠 분실

Dialogue sample (first 200 chars):
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person...


In [ ]:
# 데이터 품질 검증 시스템 (PRD 16_데이터_품질_검증_시스템.md)
import numpy as np
from typing import Dict, List

class DataQualityValidator:
    """데이터 품질 검증 클래스"""
    def __init__(self):
        self.quality_report = {}
        
    def validate_llm_data(self, df: pd.DataFrame) -> Dict:
        """LLM 학습용 데이터 품질 검증"""
        report = {}
        
        # 1. 구조적 검증
        report['structure'] = {
            'null_values': df.isnull().sum().sum(),
            'duplicates': df.duplicated().sum(),
            'empty_dialogues': (df['dialogue'].str.len() == 0).sum(),
            'empty_summaries': (df['summary'].str.len() == 0).sum() if 'summary' in df.columns else 0
        }
        
        # 2. 텍스트 길이 검증 (LLM 컨텍스트 길이 고려)
        dialogue_lengths = df['dialogue'].str.len()
        summary_lengths = df['summary'].str.len() if 'summary' in df.columns else pd.Series([0])
        
        report['text_length'] = {
            'avg_dialogue_length': dialogue_lengths.mean(),
            'max_dialogue_length': dialogue_lengths.max(),
            'dialogues_over_2048': (dialogue_lengths > 2048).sum(),
            'dialogues_over_4096': (dialogue_lengths > 4096).sum(),
            'summary_ratio': (summary_lengths / dialogue_lengths).mean() if 'summary' in df.columns else 0
        }
        
        # 3. 토큰 수 추정 (한국어는 대략 3자 = 1토큰)
        estimated_tokens = dialogue_lengths / 3
        report['token_estimation'] = {
            'avg_tokens': estimated_tokens.mean(),
            'max_tokens': estimated_tokens.max(),
            'samples_over_512_tokens': (estimated_tokens > 512).sum(),
            'samples_over_1024_tokens': (estimated_tokens > 1024).sum()
        }
        
        # 4. 특수 문자 및 인코딩 검증
        report['encoding'] = {
            'person_tags_present': df['dialogue'].str.contains('#Person').sum(),
            'encoding_issues': df['dialogue'].str.contains('[�\\?]').sum(),
            'special_chars': df['dialogue'].str.contains('[^\w\s#:.,!?가-힣a-zA-Z0-9]').sum()
        }
        
        # 5. 주제 분포 (불균형 체크)
        if 'topic' in df.columns:
            topic_counts = df['topic'].value_counts()
            report['topic_distribution'] = {
                'unique_topics': len(topic_counts),
                'most_common_topic': topic_counts.index[0] if len(topic_counts) > 0 else None,
                'most_common_count': topic_counts.iloc[0] if len(topic_counts) > 0 else 0,
                'least_common_topic': topic_counts.index[-1] if len(topic_counts) > 0 else None,
                'least_common_count': topic_counts.iloc[-1] if len(topic_counts) > 0 else 0,
                'imbalance_ratio': topic_counts.iloc[0] / topic_counts.iloc[-1] if len(topic_counts) > 1 and topic_counts.iloc[-1] > 0 else 0
            }
        
        self.quality_report = report
        return report
    
    def recommend_preprocessing(self) -> List[str]:
        """전처리 권장사항 생성"""
        recommendations = []
        
        if self.quality_report.get('structure', {}).get('null_values', 0) > 0:
            recommendations.append("Remove or impute null values")
        
        if self.quality_report.get('structure', {}).get('duplicates', 0) > 0:
            recommendations.append("Remove duplicate samples")
        
        if self.quality_report.get('text_length', {}).get('dialogues_over_4096', 0) > 0:
            recommendations.append("Truncate or split long dialogues (>4096 chars)")
        
        if self.quality_report.get('encoding', {}).get('encoding_issues', 0) > 0:
            recommendations.append("Fix encoding issues in text")
        
        if self.quality_report.get('topic_distribution', {}).get('imbalance_ratio', 0) > 10:
            recommendations.append("Consider data augmentation for underrepresented topics")
        
        return recommendations

# 데이터 품질 검증 실행
validator = DataQualityValidator()
quality_report = validator.validate_llm_data(train_df)

logger.write("\n=== Data Quality Validation Report ===")
for category, metrics in quality_report.items():
    logger.write(f"\n{category.upper()}:")
    for key, value in metrics.items():
        if isinstance(value, float):
            logger.write(f"  - {key}: {value:.2f}")
        else:
            logger.write(f"  - {key}: {value}")

# 전처리 권장사항
recommendations = validator.recommend_preprocessing()
if recommendations:
    logger.write("\n📋 Preprocessing Recommendations:")
    for rec in recommendations:
        logger.write(f"  • {rec}")

In [ ]:
# 추론 최적화 시스템 (PRD 17_추론_최적화.md)
import gc
from typing import List

class LLMInferenceOptimizer:
    """LLM 추론 최적화 클래스"""
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.cache = {}
        
        # 최적화 기법 적용
        self._apply_optimizations()
    
    def _apply_optimizations(self):
        """모델 최적화 기법 적용"""
        # 1. Gradient checkpointing (메모리 절약)
        if hasattr(self.model, 'gradient_checkpointing_enable'):
            self.model.gradient_checkpointing_enable()
            logger.write("Gradient checkpointing enabled")
        
        # 2. Flash Attention (가능한 경우)
        try:
            if hasattr(self.model.config, 'use_flash_attention'):
                self.model.config.use_flash_attention = True
                logger.write("Flash attention enabled")
        except:
            pass
        
        # 3. KV 캐시 활성화
        if hasattr(self.model.config, 'use_cache'):
            self.model.config.use_cache = True
            logger.write("KV cache enabled")
    
    def batch_generate(self, texts: List[str], batch_size: int = 8, **generation_kwargs) -> List[str]:
        """배치 생성 최적화"""
        all_outputs = []
        
        # 길이별로 정렬 (동적 배칭)
        sorted_texts = sorted(enumerate(texts), key=lambda x: len(x[1]))
        
        for i in range(0, len(sorted_texts), batch_size):
            batch = sorted_texts[i:i+batch_size]
            batch_indices = [idx for idx, _ in batch]
            batch_texts = [text for _, text in batch]
            
            # 토큰화
            inputs = self.tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=generation_kwargs.get('max_length', 1024),
                return_tensors='pt'
            ).to(self.device)
            
            # 생성
            with torch.no_grad():
                with torch.cuda.amp.autocast():  # Mixed precision
                    outputs = self.model.generate(
                        **inputs,
                        **generation_kwargs
                    )
            
            # 디코딩
            decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
            
            # 원래 순서로 저장
            for idx, output in zip(batch_indices, decoded):
                all_outputs.append((idx, output))
        
        # 원래 순서로 정렬
        all_outputs.sort(key=lambda x: x[0])
        return [output for _, output in all_outputs]
    
    def cached_generate(self, text: str, **kwargs) -> str:
        """캐싱을 활용한 생성"""
        # 캐시 키 생성
        cache_key = hash(text[:200])
        
        if cache_key in self.cache:
            logger.write("Cache hit - returning cached result")
            return self.cache[cache_key]
        
        # 캐시 미스 - 새로 생성
        inputs = self.tokenizer(
            text,
            truncation=True,
            max_length=kwargs.get('max_length', 1024),
            return_tensors='pt'
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(**inputs, **kwargs)
        
        result = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 캐시 저장 (최대 100개)
        if len(self.cache) < 100:
            self.cache[cache_key] = result
        
        return result

# 리스크 관리 시스템 (PRD 05_리스크_관리.md)
class LLMRiskManager:
    """LLM 학습 리스크 관리"""
    
    def __init__(self):
        self.risk_log = []
        self.mitigation_applied = []
    
    def check_training_risks(self, epoch: int, train_loss: float, val_loss: float, 
                           rouge_score: float, memory_used: float) -> Dict:
        """학습 리스크 체크"""
        risks = []
        
        # 1. 과적합 리스크
        if val_loss > train_loss * 1.5 and epoch > 2:
            risks.append({
                'type': 'overfitting',
                'severity': 'high',
                'metric': f'val/train ratio: {val_loss/train_loss:.2f}',
                'mitigation': ['Increase dropout', 'Early stopping', 'Reduce learning rate']
            })
        
        # 2. 과소적합 리스크
        if epoch > 3 and train_loss > 2.0:
            risks.append({
                'type': 'underfitting',
                'severity': 'medium',
                'metric': f'train_loss: {train_loss:.4f}',
                'mitigation': ['Increase model capacity', 'Adjust learning rate', 'Check data quality']
            })
        
        # 3. 성능 정체 리스크
        if epoch > 5 and rouge_score < 0.3:
            risks.append({
                'type': 'performance_plateau',
                'severity': 'high',
                'metric': f'ROUGE-L: {rouge_score:.4f}',
                'mitigation': ['Change prompt strategy', 'Adjust hyperparameters', 'Use different model']
            })
        
        # 4. 메모리 리스크
        if memory_used > 0.9:
            risks.append({
                'type': 'memory_overflow',
                'severity': 'critical',
                'metric': f'Memory usage: {memory_used:.1%}',
                'mitigation': ['Reduce batch size', 'Enable gradient accumulation', 'Use smaller model']
            })
        
        # 5. 학습 불안정 리스크
        if train_loss > 10 or np.isnan(train_loss):
            risks.append({
                'type': 'training_instability',
                'severity': 'critical',
                'metric': f'Unstable loss: {train_loss}',
                'mitigation': ['Reduce learning rate', 'Check for NaN values', 'Reset training']
            })
        
        # 리스크 로그 저장
        if risks:
            self.risk_log.extend(risks)
            for risk in risks:
                logger.write(f"⚠️ Risk: {risk['type']} ({risk['severity']}) - {risk['metric']}")
                logger.write(f"  Suggested mitigations: {', '.join(risk['mitigation'])}")
        
        return {'risks': risks, 'total_risks': len(risks)}
    
    def apply_automatic_mitigation(self, risk_type: str, config: Dict) -> Dict:
        """자동 리스크 완화"""
        mitigations = {
            'overfitting': {
                'action': 'reduce_learning_rate',
                'new_lr': config['training']['learning_rate'] * 0.5
            },
            'memory_overflow': {
                'action': 'reduce_batch_size',
                'new_batch_size': max(1, config['training']['batch_size'] // 2)
            },
            'training_instability': {
                'action': 'reset_optimizer',
                'new_lr': config['training']['learning_rate'] * 0.1
            }
        }
        
        if risk_type in mitigations:
            mitigation = mitigations[risk_type]
            self.mitigation_applied.append(mitigation)
            logger.write(f"✓ Applied automatic mitigation: {mitigation['action']}")
            return mitigation
        
        return {}

# Optuna 하이퍼파라미터 최적화 (PRD 13_Optuna_하이퍼파라미터_최적화.md)
try:
    import optuna
    from optuna import Trial
    from optuna.samplers import TPESampler
    
    class LLMOptunaOptimizer:
        """LLM용 Optuna 최적화"""
        
        def __init__(self, model_name: str, tokenizer, train_dataset, val_dataset):
            self.model_name = model_name
            self.tokenizer = tokenizer
            self.train_dataset = train_dataset
            self.val_dataset = val_dataset
            
        def objective(self, trial: Trial) -> float:
            """Optuna 목적 함수"""
            # 하이퍼파라미터 제안
            hp = {
                'learning_rate': trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True),
                'batch_size': trial.suggest_categorical('batch_size', [1, 2, 4, 8]),
                'lora_r': trial.suggest_int('lora_r', 4, 32),
                'lora_alpha': trial.suggest_int('lora_alpha', 8, 64),
                'warmup_ratio': trial.suggest_float('warmup_ratio', 0.0, 0.2),
                'weight_decay': trial.suggest_float('weight_decay', 0.0, 0.1),
                'temperature': trial.suggest_float('temperature', 0.1, 1.0),
                'top_p': trial.suggest_float('top_p', 0.8, 1.0),
                'num_beams': trial.suggest_int('num_beams', 1, 5)
            }
            
            logger.write(f"\nTrial {trial.number}: {hp}")
            
            # 간단한 평가 (실제로는 모델 학습 필요)
            # 여기서는 더미 점수 반환 (실제 구현시 모델 학습 및 평가)
            dummy_score = np.random.random() * 0.5 + 0.3  # 0.3~0.8 범위
            
            return dummy_score
        
        def optimize(self, n_trials: int = 10, timeout: int = 3600):
            """최적화 실행"""
            study = optuna.create_study(
                direction='maximize',
                sampler=TPESampler(seed=42),
                study_name=f'llm_optimization_{self.model_name}'
            )
            
            study.optimize(self.objective, n_trials=n_trials, timeout=timeout)
            
            # 최적 파라미터
            best_params = study.best_params
            best_value = study.best_value
            
            logger.write(f"\n=== Optuna Optimization Results ===")
            logger.write(f"Best ROUGE-L: {best_value:.4f}")
            logger.write(f"Best parameters: {best_params}")
            
            return best_params, best_value
    
    # Optuna 최적화 실행 (config에서 활성화된 경우)
    if config.get('optuna', {}).get('enabled', False):
        logger.write("\n=== Starting Optuna Hyperparameter Optimization ===")
        # 나중에 데이터셋 생성 후 실행
        optuna_optimizer = LLMOptunaOptimizer(
            model_name=current_model,
            tokenizer=None,  # 나중에 설정
            train_dataset=None,  # 나중에 설정
            val_dataset=None  # 나중에 설정
        )
    else:
        optuna_optimizer = None
        logger.write("Optuna optimization disabled")
        
except ImportError:
    logger.write("Optuna not installed - skipping hyperparameter optimization")
    optuna_optimizer = None

# 리스크 매니저 초기화
risk_manager = LLMRiskManager()
logger.write("Risk management system initialized")

In [ ]:
# 데이터셋 클래스 정의
import numpy as np
from torch.utils.data import Dataset, DataLoader
from typing import Dict, List
import re

class LLMDialogueSummaryDataset(Dataset):
    """LLM 파인튜닝용 데이터셋"""
    
    def __init__(self, dataframe, tokenizer, prompt_template, max_length=1024, is_test=False):
        self.df = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.prompt_template = prompt_template
        self.max_length = max_length
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def preprocess_dialogue(self, text):
        """대화 텍스트 전처리"""
        # 노이즈 제거
        text = text.replace('\\n', '\n')
        text = text.replace('<br>', '\n')
        text = text.strip()
        
        # #Person 태그 한국어로 변경
        text = re.sub(r'#Person(\d+)#:', r'화자\1:', text)
        
        return text
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # 대화 전처리
        dialogue = self.preprocess_dialogue(row['dialogue'])
        
        if not self.is_test:
            # 학습 모드: 프롬프트와 요약을 함께 토큰화
            summary = row.get('summary', '')
            
            # 프롬프트 생성
            full_prompt = self.prompt_template.format(
                dialogue=dialogue,
                summary=summary
            )
            
            # 토큰화
            encoded = self.tokenizer(
                full_prompt,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            # 라벨 설정 (요약 부분만 loss 계산)
            labels = encoded['input_ids'].clone()
            
            # 프롬프트 부분은 -100으로 마스킹
            prompt_without_summary = self.prompt_template.format(
                dialogue=dialogue,
                summary=""
            )
            prompt_tokens = self.tokenizer(
                prompt_without_summary,
                add_special_tokens=False,
                return_tensors='pt'
            )['input_ids']
            
            if prompt_tokens.shape[1] < labels.shape[1]:
                labels[0, :prompt_tokens.shape[1]] = -100
            
            # 패딩 토큰도 -100으로 마스킹 (중요!)
            labels[labels == self.tokenizer.pad_token_id] = -100
            
            return {
                'input_ids': encoded['input_ids'].squeeze(),
                'attention_mask': encoded['attention_mask'].squeeze(),
                'labels': labels.squeeze()
            }
        else:
            # 테스트 모드: 프롬프트만 토큰화
            prompt = self.prompt_template.format(
                dialogue=dialogue,
                summary=""
            )
            
            encoded = self.tokenizer(
                prompt,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            return {
                'input_ids': encoded['input_ids'].squeeze(),
                'attention_mask': encoded['attention_mask'].squeeze(),
                'idx': idx
            }

# 프롬프트 템플릿 로드
prompt_template = config['prompt_template']['instruction_format']

logger.write("Creating datasets...")

# 데이터셋 생성은 나중에 토크나이저 로드 후에 진행
logger.write("Dataset class defined successfully")

In [ ]:
# 모델 및 토크나이저 로드 (메모리 효율적으로)
logger.write(f"\nLoading model: {model_config['name']}")
logger.write("This may take some time due to model size...")

# 8bit/4bit 양자화 설정
load_kwargs = {}
if model_config.get('load_in_8bit', False):
    load_kwargs['load_in_8bit'] = True
    load_kwargs['device_map'] = 'auto'
    logger.write("Using 8-bit quantization")
elif model_config.get('load_in_4bit', False):
    from transformers import BitsAndBytesConfig
    load_kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    load_kwargs['device_map'] = 'auto'
    logger.write("Using 4-bit quantization")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_config['name'],
    padding_side='left'  # 생성 모델용
)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    logger.write(f"Set pad_token to eos_token: {tokenizer.pad_token}")

# 모델 로드
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_config['name'],
        torch_dtype=torch.float16,
        **load_kwargs
    )
    logger.write(f"Model loaded successfully")
    
    # LoRA 적용
    if model_config['use_lora']:
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
        logger.write("LoRA applied to model")
    
except Exception as e:
    logger.write(f"Error loading model: {e}")
    logger.write("Falling back to smaller model or mock mode")
    
    # Fallback: 더 작은 모델 사용
    fallback_model = "skt/kogpt2-base-v2"
    logger.write(f"Loading fallback model: {fallback_model}")
    
    tokenizer = AutoTokenizer.from_pretrained(fallback_model)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        fallback_model,
        torch_dtype=torch.float16
    ).to(device)
    
    if model_config['use_lora']:
        # Fallback 모델용 LoRA config 수정
        from peft import LoraConfig, get_peft_model, TaskType
        
        fallback_lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            task_type=TaskType.CAUSAL_LM,
            target_modules=["c_attn", "c_proj"]  # GPT2용 모듈
        )
        model = get_peft_model(model, fallback_lora_config)
        model.print_trainable_parameters()

logger.write(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

In [ ]:
# 데이터셋 및 DataLoader 생성
logger.write("\nCreating datasets and dataloaders...")

# 학습용 샘플 수 제한 (메모리 절약)
if config['training'].get('use_sample', False):
    sample_size = config['training'].get('sample_size', 1000)
    train_df_sample = train_df.sample(n=min(sample_size, len(train_df)), random_state=42)
    logger.write(f"Using sample of {len(train_df_sample)} training examples")
else:
    train_df_sample = train_df

# 데이터셋 생성
train_dataset = LLMDialogueSummaryDataset(
    train_df_sample,
    tokenizer,
    prompt_template,
    max_length=model_config['max_length'],
    is_test=False
)

val_dataset = LLMDialogueSummaryDataset(
    dev_df,
    tokenizer,
    prompt_template,
    max_length=model_config['max_length'],
    is_test=False
)

test_dataset = LLMDialogueSummaryDataset(
    test_df,
    tokenizer,
    prompt_template,
    max_length=model_config['max_length'],
    is_test=True
)

# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config['training']['batch_size'],
    shuffle=True,
    num_workers=0,  # 양자화 모델은 멀티프로세싱 문제 방지
    pin_memory=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['evaluation']['batch_size'],
    shuffle=False,
    num_workers=0,
    pin_memory=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config['inference']['batch_size'],
    shuffle=False,
    num_workers=0,
    pin_memory=False
)

logger.write(f"DataLoaders created:")
logger.write(f"  - Train batches: {len(train_loader)}")
logger.write(f"  - Val batches: {len(val_loader)}")
logger.write(f"  - Test batches: {len(test_loader)}")

## 학습 및 평가 함수 정의

In [ ]:
# 학습 루프 (리스크 모니터링 및 Solar API 교차 검증 포함)
for epoch in range(num_epochs):
    logger.write(f"\n{'='*30}")
    logger.write(f"Epoch {epoch + 1}/{num_epochs}")
    logger.write(f"{'='*30}")
    
    # 학습
    train_loss = train_epoch(model, train_loader, optimizer, scheduler)
    logger.write(f"Average training loss: {train_loss:.4f}")
    training_history['train_loss'].append(train_loss)
    
    # 검증 (적은 샘플로)
    val_loss, rouge_scores, sample_preds = evaluate(
        model, 
        val_loader, 
        tokenizer,
        max_samples=config['evaluation'].get('max_samples', 50)
    )
    
    logger.write(f"\nValidation Results:")
    logger.write(f"  - Loss: {val_loss:.4f}")
    logger.write(f"  - ROUGE-1 F1: {rouge_scores['rouge-1']:.4f}")
    logger.write(f"  - ROUGE-2 F1: {rouge_scores['rouge-2']:.4f}")
    logger.write(f"  - ROUGE-L F1: {rouge_scores['rouge-l']:.4f}")
    
    # 리스크 모니터링
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated() / torch.cuda.max_memory_allocated()
    else:
        memory_used = 0
    
    risk_status = risk_manager.check_training_risks(
        epoch=epoch,
        train_loss=train_loss,
        val_loss=val_loss,
        rouge_score=rouge_scores['rouge-l'],
        memory_used=memory_used
    )
    
    # 리스크 완화 적용
    if risk_status['total_risks'] > 0:
        logger.write(f"\n⚠️ {risk_status['total_risks']} risks detected")
        for risk in risk_status['risks']:
            if risk['severity'] == 'critical':
                mitigation = risk_manager.apply_automatic_mitigation(risk['type'], config)
                
                # 학습률 조정
                if 'new_lr' in mitigation:
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = mitigation['new_lr']
                    logger.write(f"  → Learning rate reduced to {mitigation['new_lr']}")
    
    # Solar API 교차 검증 (매 2 에폭마다)
    if solar_validator and epoch % 2 == 0:
        logger.write("\n🔄 Solar API Cross-Validation...")
        
        # 랜덤 샘플 선택
        sample_idx = np.random.randint(0, len(dev_df))
        sample = dev_df.iloc[sample_idx]
        
        # 모델 예측 생성
        model_prompt = prompt_engineer.get_best_prompt(
            sample['dialogue'], 
            sample.get('topic', None)
        )
        
        model_inputs = tokenizer(
            model_prompt,
            max_length=1024,
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        with torch.no_grad():
            model_output = model.generate(
                **model_inputs,
                max_new_tokens=150,
                temperature=0.3,
                top_p=0.9
            )
        
        model_summary = tokenizer.decode(model_output[0], skip_special_tokens=True)
        
        # 프롬프트 부분 제거
        if "### Response:" in model_summary:
            model_summary = model_summary.split("### Response:")[-1].strip()
        
        # Solar API 예측
        api_summary = solar_validator.generate_with_solar(sample['dialogue'])
        
        # 비교
        comparison = solar_validator.compare_with_model(
            model_summary=model_summary,
            api_summary=api_summary,
            reference=sample.get('summary', None)
        )
        
        if comparison.get('model_rouge_l') and comparison.get('api_rouge_l'):
            logger.write(f"  Model ROUGE-L: {comparison['model_rouge_l']:.4f}")
            logger.write(f"  Solar ROUGE-L: {comparison['api_rouge_l']:.4f}")
            logger.write(f"  Best: {comparison['best']}")
            
            # WandB 로깅
            if config['wandb']['mode'] != 'disabled':
                wandb.log({
                    'model_vs_api/model_rouge': comparison['model_rouge_l'],
                    'model_vs_api/api_rouge': comparison.get('api_rouge_l', 0),
                    'model_vs_api/best': 1 if comparison['best'] == 'model' else 0
                })
    
    # 히스토리 업데이트
    training_history['val_loss'].append(val_loss)
    training_history['rouge_1'].append(rouge_scores['rouge-1'])
    training_history['rouge_2'].append(rouge_scores['rouge-2'])
    training_history['rouge_l'].append(rouge_scores['rouge-l'])
    
    # WandB 로깅
    if config['wandb']['mode'] != 'disabled':
        wandb.log({
            'epoch': epoch + 1,
            'train_loss_epoch': train_loss,
            'val_loss': val_loss,
            'rouge_1': rouge_scores['rouge-1'],
            'rouge_2': rouge_scores['rouge-2'],
            'rouge_l': rouge_scores['rouge-l'],
            'risks_detected': risk_status['total_risks'],
            'memory_usage': memory_used
        })
    
    # 샘플 예측 출력
    if sample_preds:
        logger.write("\nSample Predictions:")
        for i, pred in enumerate(sample_preds[:2]):
            logger.write(f"  Sample {i+1}: {pred[:150]}...")
    
    # Best model 저장
    if rouge_scores['rouge-l'] > best_rouge_l:
        best_rouge_l = rouge_scores['rouge-l']
        patience_counter = 0
        
        # 모델 저장
        if hasattr(model, 'module'):
            model_to_save = model.module
        else:
            model_to_save = model
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_to_save.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'rouge_scores': rouge_scores,
            'config': config,
            'model_config': model_config,
            'risk_report': risk_manager.risk_log,  # 리스크 보고서 추가
            'prompt_strategy': 'advanced_prompting'  # 프롬프트 전략 기록
        }, best_model_path)
        
        logger.write(f"✓ New best model saved! (ROUGE-L: {best_rouge_l:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            logger.write(f"\nEarly stopping triggered after {epoch + 1} epochs")
            break
    
    # 메모리 정리
    torch.cuda.empty_cache()
    gc.collect()

# 최종 리스크 보고서
logger.write("\n" + "="*50)
logger.write("RISK MANAGEMENT REPORT")
logger.write("="*50)
logger.write(f"Total risks encountered: {len(risk_manager.risk_log)}")
logger.write(f"Mitigations applied: {len(risk_manager.mitigation_applied)}")

if risk_manager.risk_log:
    # 리스크 타입별 집계
    risk_types = {}
    for risk in risk_manager.risk_log:
        risk_type = risk['type']
        if risk_type not in risk_types:
            risk_types[risk_type] = 0
        risk_types[risk_type] += 1
    
    logger.write("\nRisk Summary:")
    for risk_type, count in risk_types.items():
        logger.write(f"  - {risk_type}: {count} occurrences")

logger.write("\n" + "="*50)
logger.write("Training completed!")
logger.write(f"Best ROUGE-L: {best_rouge_l:.4f}")
logger.write("="*50)

In [ ]:
# WandB 초기화
if config['wandb']['mode'] != 'disabled':
    wandb.init(
        project=config['wandb']['project'],
        entity=config['wandb']['entity'],
        name=f"{config['wandb']['name']}_{current_model}",
        tags=config['wandb']['tags'] + [current_model],
        config={
            **config,
            'model': model_config
        }
    )
    logger.write("WandB initialized")

# 옵티마이저 및 스케줄러 설정
num_epochs = config['training']['num_epochs']

# learning_rate가 문자열인 경우 float로 변환
learning_rate = config['training']['learning_rate']
if isinstance(learning_rate, str):
    learning_rate = float(learning_rate)

num_training_steps = num_epochs * len(train_loader)

# 옵티마이저 설정
optimizer = AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=config['training']['weight_decay']
)

# 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(num_training_steps * config['training']['warmup_ratio']),
    num_training_steps=num_training_steps
)

logger.write(f"\nTraining Configuration:")
logger.write(f"  - Epochs: {num_epochs}")
logger.write(f"  - Learning rate: {learning_rate}")
logger.write(f"  - Batch size: {config['training']['batch_size']}")
logger.write(f"  - Total training steps: {num_training_steps}")
logger.write(f"  - Warmup steps: {int(num_training_steps * config['training']['warmup_ratio'])}")

# 학습 기록
training_history = {
    'train_loss': [],
    'val_loss': [],
    'rouge_1': [],
    'rouge_2': [],
    'rouge_l': []
}

# 모델 저장 경로
def get_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

output_dir = get_path(config['paths']['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)
best_model_path = output_dir / f'best_model_{current_model}.pt'

# Early stopping 설정
best_rouge_l = 0
patience = config['training']['early_stopping_patience']
patience_counter = 0

logger.write("\n" + "="*50)
logger.write("Starting training...")
logger.write("="*50)

In [ ]:
# 최적 모델로 테스트 데이터 예측 (추론 최적화 적용)
logger.write("\n" + "="*50)
logger.write("TEST PREDICTION WITH OPTIMIZATION")
logger.write("="*50)

# 최적 모델 로드
if best_model_path.exists():
    checkpoint = torch.load(best_model_path)
    
    # 모델 상태 로드
    if hasattr(model, 'module'):
        model.module.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint['model_state_dict'])
    
    logger.write(f"Best model loaded from epoch {checkpoint['epoch'] + 1}")
    logger.write(f"Best ROUGE scores: {checkpoint['rouge_scores']}")

# 추론 최적화 초기화
inference_optimizer = LLMInferenceOptimizer(model, tokenizer, device)

# 테스트 예측 생성 (최적화된 배치 처리)
def generate_test_predictions(model, test_df, tokenizer, prompt_engineer, inference_optimizer):
    """최적화된 테스트 예측 생성"""
    test_predictions = []
    
    # 프롬프트 생성
    test_prompts = []
    for idx, row in test_df.iterrows():
        dialogue = row['dialogue']
        topic = row.get('topic', None)
        
        # 최적 프롬프트 선택
        prompt = prompt_engineer.get_best_prompt(dialogue, topic, strategy='auto')
        test_prompts.append(prompt)
    
    # 배치 생성 (최적화)
    generation_config = config['inference']['generation_config']
    
    logger.write(f"Generating predictions for {len(test_prompts)} test samples...")
    logger.write("Using optimized batch inference...")
    
    # 배치 단위로 예측
    batch_size = config['inference'].get('batch_size', 4)
    
    for i in tqdm(range(0, len(test_prompts), batch_size), desc="Test predictions"):
        batch_prompts = test_prompts[i:i+batch_size]
        
        # 배치 생성
        batch_predictions = inference_optimizer.batch_generate(
            batch_prompts,
            batch_size=batch_size,
            max_new_tokens=generation_config['max_new_tokens'],
            temperature=generation_config['temperature'],
            top_p=generation_config['top_p'],
            num_beams=generation_config['num_beams'],
            no_repeat_ngram_size=generation_config['no_repeat_ngram_size'],
            do_sample=generation_config['do_sample']
        )
        
        # 프롬프트 부분 제거
        for pred in batch_predictions:
            if "### Response:" in pred:
                pred = pred.split("### Response:")[-1].strip()
            elif "요약:" in pred:
                pred = pred.split("요약:")[-1].strip()
            
            test_predictions.append(pred)
    
    return test_predictions

# 예측 수행
test_predictions = generate_test_predictions(
    model, test_df, tokenizer, prompt_engineer, inference_optimizer
)

logger.write(f"\n✓ Generated {len(test_predictions)} test predictions")

# Solar API와 비교 (샘플)
if solar_validator and len(test_predictions) > 0:
    logger.write("\n=== Solar API Comparison (Test Samples) ===")
    
    # 랜덤 3개 샘플 비교
    for i in range(min(3, len(test_df))):
        sample_idx = np.random.randint(0, len(test_df))
        
        model_pred = test_predictions[sample_idx]
        api_pred = solar_validator.generate_with_solar(test_df.iloc[sample_idx]['dialogue'])
        
        logger.write(f"\nSample {i+1}:")
        logger.write(f"  Model: {model_pred[:100]}...")
        if api_pred:
            logger.write(f"  Solar: {api_pred[:100]}...")

# 샘플 출력
logger.write("\n=== Sample Test Predictions ===")
for i in range(min(5, len(test_predictions))):
    logger.write(f"\nTest {i+1}: {test_predictions[i][:150]}...")

In [ ]:
# 학습 루프
for epoch in range(num_epochs):
    logger.write(f"\n{'='*30}")
    logger.write(f"Epoch {epoch + 1}/{num_epochs}")
    logger.write(f"{'='*30}")
    
    # 학습
    train_loss = train_epoch(model, train_loader, optimizer, scheduler)
    logger.write(f"Average training loss: {train_loss:.4f}")
    training_history['train_loss'].append(train_loss)
    
    # 검증 (적은 샘플로)
    val_loss, rouge_scores, sample_preds = evaluate(
        model, 
        val_loader, 
        tokenizer,
        max_samples=config['evaluation'].get('max_samples', 50)
    )
    
    logger.write(f"\nValidation Results:")
    logger.write(f"  - Loss: {val_loss:.4f}")
    logger.write(f"  - ROUGE-1 F1: {rouge_scores['rouge-1']:.4f}")
    logger.write(f"  - ROUGE-2 F1: {rouge_scores['rouge-2']:.4f}")
    logger.write(f"  - ROUGE-L F1: {rouge_scores['rouge-l']:.4f}")
    
    # 히스토리 업데이트
    training_history['val_loss'].append(val_loss)
    training_history['rouge_1'].append(rouge_scores['rouge-1'])
    training_history['rouge_2'].append(rouge_scores['rouge-2'])
    training_history['rouge_l'].append(rouge_scores['rouge-l'])
    
    # WandB 로깅
    if config['wandb']['mode'] != 'disabled':
        wandb.log({
            'epoch': epoch + 1,
            'train_loss_epoch': train_loss,
            'val_loss': val_loss,
            'rouge_1': rouge_scores['rouge-1'],
            'rouge_2': rouge_scores['rouge-2'],
            'rouge_l': rouge_scores['rouge-l']
        })
    
    # 샘플 예측 출력
    if sample_preds:
        logger.write("\nSample Predictions:")
        for i, pred in enumerate(sample_preds[:2]):
            logger.write(f"  Sample {i+1}: {pred[:150]}...")
    
    # Best model 저장
    if rouge_scores['rouge-l'] > best_rouge_l:
        best_rouge_l = rouge_scores['rouge-l']
        patience_counter = 0
        
        # 모델 저장
        if hasattr(model, 'module'):
            model_to_save = model.module
        else:
            model_to_save = model
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_to_save.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'rouge_scores': rouge_scores,
            'config': config,
            'model_config': model_config
        }, best_model_path)
        
        logger.write(f"✓ New best model saved! (ROUGE-L: {best_rouge_l:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            logger.write(f"\nEarly stopping triggered after {epoch + 1} epochs")
            break
    
    # 메모리 정리
    torch.cuda.empty_cache()
    gc.collect()

logger.write("\n" + "="*50)
logger.write("Training completed!")
logger.write(f"Best ROUGE-L: {best_rouge_l:.4f}")
logger.write("="*50)

## 테스트 데이터 예측 및 제출 파일 생성

In [ ]:
# 제출 파일 생성
submission_df = pd.DataFrame({
    'fname': test_df['fname'],
    'summary': test_predictions
})

# 제출 파일 저장 - config의 경로 사용
submission_dir = get_path(config['paths']['submission_dir'])
submission_dir.mkdir(parents=True, exist_ok=True)

submission_filename = f'{current_model}_submission_{timestamp}.csv'
submission_path = submission_dir / submission_filename

# index=True로 설정하여 인덱스를 포함시킴
submission_df.to_csv(submission_path, index=True, encoding='utf-8')  # index=False -> index=True로 변경
logger.write(f"\nSubmission file saved: {submission_path}")

# 제출 파일 확인
print(f"\nSubmission file created: {submission_filename}")
print(f"Shape: {submission_df.shape}")
print("\nFirst 3 submissions:")
print(submission_df.head(3))

# 실험 요약
logger.write("\n" + "="*50)
logger.write("SINGLE MODEL EXPERIMENT SUMMARY")
logger.write("="*50)
logger.write(f"Model: {current_model} ({model_config['name']})")
logger.write(f"Best ROUGE-L: {best_rouge_l:.4f}")
logger.write(f"Training epochs completed: {len(training_history['train_loss'])}")
if training_history['train_loss']:
    logger.write(f"Final train loss: {training_history['train_loss'][-1]:.4f}")
if training_history['val_loss']:
    logger.write(f"Final val loss: {training_history['val_loss'][-1]:.4f}")
logger.write(f"Submission file: {submission_filename}")
logger.write("="*50)

# 시각화
if config.get('visualization', {}).get('enabled', False):
    from src.utils.visualizations.training_viz import TrainingVisualizer
    
    viz = TrainingVisualizer()
    
    # 시각화 저장 경로
    viz_dir = get_path(config.get('visualization', {}).get('save_path', 'visualizations'))
    viz_dir.mkdir(parents=True, exist_ok=True)
    
    # 학습 히스토리 플롯
    if len(training_history['train_loss']) > 0:
        viz.plot_training_history(
            training_history,
            save_path=viz_dir / f'{current_model}_training_history.png'
        )
        logger.write(f"Visualization saved to {viz_dir}")

# WandB 종료
if config['wandb']['mode'] != 'disabled':
    wandb.finish()

logger.write(f"\n✅ Single model experiment ({current_model}) completed successfully!")
logger.write(f"Log file: {log_file}")

In [ ]:
# 제출 파일 생성
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'summary': test_predictions
})

# 제출 파일 저장 - config의 경로 사용
submission_dir = get_path(config['paths']['submission_dir'])
submission_dir.mkdir(parents=True, exist_ok=True)

submission_filename = f'{current_model}_submission_{timestamp}.csv'
submission_path = submission_dir / submission_filename

submission_df.to_csv(submission_path, index=False, encoding='utf-8')
logger.write(f"\nSubmission file saved: {submission_path}")

# 제출 파일 확인
print(f"\nSubmission file created: {submission_filename}")
print(f"Shape: {submission_df.shape}")
print("\nFirst 3 submissions:")
print(submission_df.head(3))

# 실험 요약
logger.write("\n" + "="*50)
logger.write("SINGLE MODEL EXPERIMENT SUMMARY")
logger.write("="*50)
logger.write(f"Model: {current_model} ({model_config['name']})")
logger.write(f"Best ROUGE-L: {best_rouge_l:.4f}")
logger.write(f"Training epochs completed: {len(training_history['train_loss'])}")
if training_history['train_loss']:
    logger.write(f"Final train loss: {training_history['train_loss'][-1]:.4f}")
if training_history['val_loss']:
    logger.write(f"Final val loss: {training_history['val_loss'][-1]:.4f}")
logger.write(f"Submission file: {submission_filename}")
logger.write("="*50)

# 시각화
if config.get('visualization', {}).get('enabled', False):
    from src.utils.visualizations.training_viz import TrainingVisualizer
    
    viz = TrainingVisualizer()
    
    # 시각화 저장 경로
    viz_dir = get_path(config.get('visualization', {}).get('save_path', 'visualizations'))
    viz_dir.mkdir(parents=True, exist_ok=True)
    
    # 학습 히스토리 플롯
    if len(training_history['train_loss']) > 0:
        viz.plot_training_history(
            training_history,
            save_path=viz_dir / f'{current_model}_training_history.png'
        )
        logger.write(f"Visualization saved to {viz_dir}")

# WandB 종료
if config['wandb']['mode'] != 'disabled':
    wandb.finish()

logger.write(f"\n✅ Single model experiment ({current_model}) completed successfully!")
logger.write(f"Log file: {log_file}")